In [1]:
%reload_ext autoreload
%autoreload 2

In [8]:
from result_saver import SaverProvider

provider = SaverProvider()
backend = provider.get_backend('ibm_sherbrooke')

In [12]:
from Scratch import metadata_loader

md = metadata_loader(True, True)
md = md[md["notebook_name"]=='Torino_RepCodes_jobs']
# md = md[md["notebook_name"]=='Sherbrooke_RepCodes_jobs']
md = md[md["job_status"] == "JobStatus.DONE"]
md.dropna(subset=['xbasis', 'logical', 'rounds'], inplace=True)

print("shape before", md.shape)

unique_jobs = md.drop_duplicates(subset=['xbasis', 'logical', 'rounds'], keep='first')
# unique_jobs
print("shape after", unique_jobs.shape)

shape before (131, 28)
shape after (7, 28)


In [5]:
from tqdm import tqdm
import numpy as np

from soft_info import EvaluateBottlenecks

md_s = unique_jobs


differences = {
    "mean_gate": [],
    "min_gate": [],
    "max_gate": [],
    "mean_r": [],
    "min_r": [],
    "max_r": [],
    "mean_an": [],
    "min_an": [],
    "max_an": [],
    "fid": []
}

for job_id in tqdm(md_s["job_id"]):
    job = provider.retrieve_job(job_id)
    layout = job.initial_layouts()[0]
    layout_des = job.deserialize_layout(layout) 
    link_qubits = list(layout_des["link_qubit"].values())
    code_qubits = list(layout_des["code_qubit"].values())
    path = [index for tpl in zip(code_qubits, link_qubits) for index in tpl]

    # Get the path_info in md of that job
    md_entry = md[md["job_id"] == job_id]
    execution_date = md_entry["execution_date"].values[0]
    path_info = md_entry["path_info"].values[0]    

    evaluator = EvaluateBottlenecks(backend, date=execution_date)
    error_info = evaluator.get_error_info(path=path)
    path_info_values = path_info.values()

    # Calculate and accumulate differences for each metric
    for (metric, (value, path_value)) in zip(differences.keys(), zip(error_info, path_info_values)):
        differences[metric].append(abs(value - path_value))
    


# Calculate mean difference for each metric
mean_differences = {metric: np.mean(diffs) for metric, diffs in differences.items()}

# Print mean differences
for metric, mean_diff in mean_differences.items():
    print(f"Mean difference for {metric}: {mean_diff}")


    
    

100%|██████████| 7/7 [00:27<00:00,  3.87s/it]

Mean difference for mean_gate: 1.9014869123042122e-06
Mean difference for min_gate: 2.697495005143935e-16
Mean difference for max_gate: 3.7470027081099033e-16
Mean difference for mean_r: 0.002146217386217584
Mean difference for min_r: 0.0006285714285714205
Mean difference for max_r: 0.18491428571428573
Mean difference for mean_an: 0.0008616546416540101
Mean difference for min_an: 0.0004571428571428935
Mean difference for max_an: 0.004114285714285705
Mean difference for fid: 0.003293662237541406
